# ACO-TR-LOGISTICS: Rota Optimizasyon Prototipi (Versiyon 2.0)
Bu not defteri, tüm düzeltmeleri ve en iyi pratikleri içeren, baştan sona çalışan bir prototiptir. Amacı, Çorlu bölgesi için belirlenen duraklara en verimli turu bulan bir rota optimizasyonu gerçekleştirmektir.

### Adım 1: Kurulum ve Kütüphaneleri Yükleme
Bu hücre, projenin çalışması için gerekli olan standart Python kütüphanelerini ve `src` klasörümüzdeki kendi yazdığımız `Ant` ve `ACOptimizer` sınıflarını import eder.

`sys.path` manipülasyonu, not defterimizin, bir üst dizindeki `src` klasörünü görmesini sağlamak için kritik bir adımdır.

In [ ]:
import sys
import os
import random

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.data_loader import load_graph
from src.visualization.map_plotter import plot_optimized_route
from src.aco import Ant, ACOptimizer

import numpy as np
from IPython.display import display

print("Kurulum tamamlandı. Kütüphaneler ve yerel modüller başarıyla import edildi.")

### Adım 2: Yol Ağını Çekme ve Hazırlama
Önceki hatalardan ders çıkardık. Algoritmanın tek yönlü yollarla kafasının karışmaması ve `KeyError` almamak için:
1. Önce standart 'drive' ağını (yönlü graf) çekeceğiz.
2. Sonra `ox.get_undirected()` fonksiyonu ile bu grafı **yönsüz (simetrik)** bir grafa çevireceğiz. Bu, eğer A'dan B'ye bir yol varsa, B'den A'ya da bir yol olduğunu garanti eder ve algoritmayı çok daha kararlı hale getirir.

In [ ]:
import osmnx as ox

place_name = 'Çorlu, Tekirdağ, Türkiye'

print("Yönlü graf indiriliyor (Görselleştirme için)...")
G_directed = ox.graph_from_place(place_name, network_type='drive')

print("Yönsüz graf oluşturuluyor (Optimizasyon için)...")
G_undirected = G_directed.to_undirected()

print("\nGraf yüklemesi tamamlandı.")
print(f"Yönlü Graf: {len(G_directed.nodes())} Düğüm, {len(G_directed.edges())} Kenar")
print(f"Yönsüz Graf: {len(G_undirected.nodes())} Düğüm, {len(G_undirected.edges())} Kenar")

### Adım 3: Optimizasyon Parametrelerini Ayarlama
Operasyon Şefi'nin strateji toplantısı! Algoritmamızın nasıl davranacağını belirleyen "ince ayar düğmelerini" burada tanımlıyoruz.

In [ ]:
ANT_COUNT = 10
ITERATIONS = 50
ALPHA = 1.0
BETA = 5.0
EVAPORATION_RATE = 0.5

### Adım 4: Problemi Tanımlama (Durakları Seçme
Karıncaların hangi problemi çözeceğini netleştirmeliyiz. Bu örnekte, haritadan rastgele **10 adet durak** seçeceğiz ve karıncalardan bu 10 durağın hepsine uğrayan en kısa turu bulmalarını isteyeceğiz. Bu, klasik bir **Gezgin Satıcı Problemi (TSP)** senaryosudur.

In [ ]:
if 'G_undirected' in locals():
    all_graph_nodes = list(G_undirected.nodes()) # Yönsüz graftan seç
    nodes_to_visit = random.sample(all_graph_nodes, 10)
    start_node = nodes_to_visit[0]
    print(f"Toplam {len(nodes_to_visit)} adet durağa uğranacak.")
    print(f"Başlangıç Durağı (Depo): {start_node}")
else:
    print("Graf yüklenemediği için problem tanımlanamıyor.")

### Adım 5: Optimizasyonu Başlatma
Tüm hazırlıklar tamam. Şimdi `ACOptimizer` nesnesini oluşturup, `run` metodunu çağırarak optimizasyon sürecini başlatıyoruz. Bu işlem, bilgisayarınızın hızına bağlı olarak birkaç dakika sürebilir.

In [ ]:
if 'G_undirected' in locals():
    ants = [Ant(G_undirected, start_node) for _ in range(ANT_COUNT)]
    optimizer = ACOptimizer(
        graph=G_undirected,
        ants=ants,
        nodes_to_visit=nodes_to_visit,
        alpha=ALPHA,
        beta=BETA,
        evaporation_rate=EVAPORATION_RATE
    )

    print("Optimizasyon başlıyor...")
    best_path, best_distance = optimizer.run(ITERATIONS)
    print("\nOptimizasyon tamamlandı!")
    print("--- SONUÇLAR ---")
    print(f"En iyi bulunan tur mesafesi: {best_distance:.2f} metre")

### Adım 6: Sonucu Haritada Görselleştirme
Sayısal sonuç güzel, ama en tatmin edici kısım, bulunan en iyi rotayı interaktif bir harita üzerinde görmektir. `ox.plot_route_folium` fonksiyonu bu iş için mükemmeldir.

In [ ]:
if 'best_path' in locals() and best_path and 'G_directed' in locals():
    print("Sonuç haritası oluşturuluyor...")
    
    route_map = plot_optimized_route(
        graph=G_directed,
        best_path=best_path,
        nodes_to_visit=nodes_to_visit,
        start_node=start_node
    )
    
    if route_map:
        display(route_map)
    else:
        print("Harita oluşturulamadı.")
else:
    print("Bir rota bulunamadığı için harita çizilemiyor.")